In [16]:
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict
from tqdm import tqdm
import scipy
import pyDOE

import torch
import torch.nn as nn

In [14]:
h=0.01
k=0.01
x = torch.arange(-np.pi, np.pi + h, h)
y = torch.arange(-np.pi, np.pi + k, k)
X = torch.stack(torch.meshgrid(x, y)).reshape(2, -1).T

# u = f(x,y)=A*sin(B*x)*sin(C*y)
A = torch.tensor([1.,])
B = torch.tensor([1.,])
C = torch.tensor([1.,])

# training data
bc1 = torch.stack(torch.meshgrid(x[0], y)).reshape(2, -1).T
bc2 = torch.stack(torch.meshgrid(x[-1], y)).reshape(2, -1).T
X_train = torch.cat([bc1, bc2])

In [15]:
print(x.shape, y.shape, X.shape, bc1.shape, bc2.shape, X_train.shape)

torch.Size([630]) torch.Size([630]) torch.Size([396900, 2]) torch.Size([630, 2]) torch.Size([630, 2]) torch.Size([1260, 2])


In [3]:
# Multi-layer Perceptron
class NN(nn.Module):
    def __init__(
        self,
        input_size,  
        hidden_size, 
        output_size, 
        depth,
        act=torch.nn.Tanh,
    ):
        super(NN, self).__init__()
        
        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', act()))
        for i in range(depth): 
            layers.append(
                (f'hidden_{i}', torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append((f'activation_{i}', act()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
class Net:
    def __init__(self, A=1.0, B=1.0, C=1.0):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        self.model = NN(
            input_size=2,
            hidden_size=100,
            output_size=1,
            depth=8,
            act=torch.nn.Tanh
        ).to(device)
        
        # 간격
        self.h = 0.01
        self.k = 0.01
        # (x,y) 좌표점
        x = torch.arange(-np.pi, np.pi + self.h, self.h)
        y = torch.arange(-np.pi, np.pi + self.k, self.k)
        self.X = torch.stack(torch.meshgrid(x, y)).reshape(2, -1).T
        
        # u = f(x,y)=A*sin(B*x)*sin(C*y)
        self.A = torch.tensor([A,]).to(device)
        self.B = torch.tensor([B,]).to(device)
        self.C = torch.tensor([C,]).to(device)
        
        # training data
        bc1 = torch.stack(torch.meshgrid(x[0], y)).reshape(2, -1).T
        bc2 = torch.stack(torch.meshgrid(x[-1], y)).reshape(2, -1).T
        self.X_train = torch.cat([bc1, bc2])
        
        f_bc1 = torch.zeros(len(bc1))
        f_bc2 = torch.zeros(len(bc2))
        self.f_train = torch.cat([f_bc1, f_bc2])
        self.f_train = self.f_train.unsqueeze(1)
        
        self.X = self.X.to(device)
        self.X_train = self.X_train.to(device)
        self.f_train = self.f_train.to(device)
        self.X.requires_grad = True
        
        self.criterion = torch.nn.MSELoss()
        self.iter = 1
        
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=5,
            tolerance_grad=1e-13, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe",
        )
        self.adam = torch.optim.Adam(self.model.parameters(), lr=0.00001)
        self.data_loss_hist = []
        self.pde_loss_hist = []
        self.total_loss_hist = []
        self.print_model_state()
        
    # this won't use
    def func(x,y,a=1,b=1,c=1):
        return a*np.sin(b*x)*np.sin(c*y)
    
    def print_model_state(self):
        print("Model's state_dict:")
        for param_tensor in self.model.state_dict():
            print(param_tensor, "\t", self.model.state_dict()[param_tensor].size())
            
    def loss_func(self):
        
        # 초기화
        self.adam.zero_grad()
        self.optimizer.zero_grad()
        
        f_pred = self.model(self.X_train)
        loss_data = self.criterion(f_pred, self.f_train)
        u = self.model(self.X)

        du_dX = torch.autograd.grad(
            inputs=self.X, 
            outputs=u, 
            grad_outputs=torch.ones_like(u), 
            retain_graph=True, 
            create_graph=True
        )[0]
        
        du_dx = du_dX[:, 0]
        du_dy = du_dX[:, 1]
        du_dXX = torch.autograd.grad(
            inputs=self.X, 
            outputs=du_dX, 
            grad_outputs=torch.ones_like(du_dX), 
            retain_graph=True, 
            create_graph=True
        )[0]
        
        du_dxx = du_dXX[:,0]
        du_dyy = du_dXX[:,1]
        
        # u = f(x,y)=A*sin(B*x)*sin(C*y)
        # fxx + fyy + A*(B^2+C^2)*f = 0
        loss_pde = self.criterion( du_dxx + du_dyy , -self.A*(self.B*self.B+self.C*self.C)*u.squeeze() )

        loss = loss_pde + loss_data
        loss.backward()
        if self.iter % 100 == 0: 
            print(f"it :{self.iter:06}, Data Loss :{loss_data.item()},\t PDE Loss :{loss_pde.item()}, \t Total Loss :{loss.item()}")
            self.total_loss_hist.append(loss.item())
            self.pde_loss_hist.append(loss_pde.item())
            self.data_loss_hist.append(loss_data.item())
        self.iter = self.iter + 1
        return loss
    
    def train(self):
        self.model.train()
        print("train with Adam optimizer")
        for i in range(1000):
            self.adam.step(self.loss_func)
        print("train with L-BFGs")
        self.optimizer.step(self.loss_func)
        
    def eval_(self):
        self.model.eval()